In [8]:
import os
import logging
import json
import time

# --- Configuration ---
RESUME_SIMULATION = True  # Set to True to resume from last checkpoint, False to restart
CHECKPOINT_FILE = "backtest_checkpoint.json"
INVENTORY_PATH = "cs2_trading/res/budapest_backtest_inventory.json"
LOG_FILE = "backtest.log"

# Set proxy for Google API
os.environ["HTTP_PROXY"] = "http://127.0.0.1:7897"
os.environ["HTTPS_PROXY"] = "http://127.0.0.1:7897"

from dotenv import load_dotenv
from datetime import datetime, timedelta
from cs2_trading.data.inventory import Inventory
from cs2_trading.data.api import InfoAPI
from cs2_trading.agents.ArtificialNewsAgent import ArtificialNewsAgent
from cs2_trading.strategy import DailyStrategy

# 1. Setup Environment
load_dotenv(override=True)

if not os.getenv("STEAM_API_KEY"):
    print("!!! WARNING: STEAM_API_KEY not found. !!!")
else:
    print("STEAM_API_KEY loaded successfully.")

info_api = InfoAPI(os.getenv("INFO_API_TOKEN"))
news_agent = ArtificialNewsAgent(news_dir="cs2_trading/res/news_artificial")

# 2. Initialize Simulation State (Resume vs Restart)
start_date = datetime(2025, 11, 11)
end_date = datetime(2025, 12, 20)
current_start_date = start_date
filemode = 'w'

if RESUME_SIMULATION and os.path.exists(CHECKPOINT_FILE) and os.path.exists(INVENTORY_PATH):
    print(f"Found checkpoint '{CHECKPOINT_FILE}'. Attempting to resume...")
    try:
        with open(CHECKPOINT_FILE, 'r') as f:
            state = json.load(f)
            last_date_str = state.get("last_completed_date")
            if last_date_str:
                last_date = datetime.strptime(last_date_str, "%Y-%m-%d")
                current_start_date = last_date + timedelta(days=1)
                print(f"Resuming from {current_start_date.strftime('%Y-%m-%d')} (Last completed: {last_date_str})")
                
                # Load Inventory
                my_inventory = Inventory.load(INVENTORY_PATH)
                print(f"Loaded inventory with {len(my_inventory.items)} items.")
                
                # Append to log
                filemode = 'a'
            else:
                print("Checkpoint file invalid. Restarting.")
                my_inventory = Inventory()
    except Exception as e:
        print(f"Error loading checkpoint: {e}. Restarting.")
        my_inventory = Inventory()
else:
    print("Starting fresh simulation...")
    if os.path.exists(INVENTORY_PATH): os.remove(INVENTORY_PATH)
    if os.path.exists(CHECKPOINT_FILE): os.remove(CHECKPOINT_FILE)
    my_inventory = Inventory()
    filemode = 'w'

# Configure Logging
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    filename=LOG_FILE,
    level=logging.INFO,
    format='%(asctime)s | %(levelname)-8s | %(message)s',
    datefmt='%H:%M:%S',
    encoding='utf-8',
    filemode=filemode
)
print(f"Logging configured to '{LOG_FILE}' (mode: {filemode})")

# 3. Initialize Strategy
strategy = DailyStrategy(
    my_inventory, 
    news_agent, 
    info_api, 
    llm_model="gemini-3-flash-preview",
    target_quantity=10, 
    max_buy_daily=3,
    save_path=INVENTORY_PATH # Pass the specific path for this backtest
)

# 4. Run Backtest Simulation
print("\n=== Starting Budapest Major Backtest ===")

# Calculate days remaining
days_remaining = (end_date - current_start_date).days + 1
previous_total_pnl = 0.0 # Note: PnL change calc might be jumpy on first resume day if we don't persist this too, but acceptable.

if days_remaining <= 0:
    print("Simulation already complete!")
else:
    for day_offset in range(days_remaining):
        current_sim_date = current_start_date + timedelta(days=day_offset)
        print(f"\n\n>>> Processing: {current_sim_date.strftime('%Y-%m-%d')} <<<")
        
        # Dynamic Strategy Adjustment
        if current_sim_date >= datetime(2025, 11, 24):
            if strategy.max_buy_daily < 8: 
                print("!!! MAJOR STARTED (Nov 24): Increasing Daily Buy Limit to 8 !!!")
                strategy.max_buy_daily = 8
        
        # Run Cycle
        try:
            strategy.run_daily_cycle(current_sim_date)
            
            # Save Checkpoint AFTER successful cycle
            with open(CHECKPOINT_FILE, 'w') as f:
                json.dump({"last_completed_date": current_sim_date.strftime("%Y-%m-%d")}, f)
                
        except Exception as e:
            print(f"!!! CRITICAL ERROR on {current_sim_date.strftime('%Y-%m-%d')}: {e}")
            logging.error(f"CRITICAL ERROR on {current_sim_date.strftime('%Y-%m-%d')}: {e}", exc_info=True)
            print("Stopping simulation. Fix the error and re-run to resume.")
            break

        # --- Daily Reporting ---
        msg_report_header = f"\n--- Daily Report: {current_sim_date.strftime('%Y-%m-%d')} ---"
        print(msg_report_header)
        logging.info(msg_report_header)
        
        current_equity = 0.0
        current_cost = 0.0
        
        header_row = f"{'Item':<30} | {'Price':<10} | {'PnL %':<8}"
        print(header_row)
        logging.info(header_row)
        
        for item in my_inventory.items:
            price = item.daily_price[-1] if item.daily_price else item.bought_price
            current_equity += price
            current_cost += item.bought_price
            pnl_pct = ((price - item.bought_price) / item.bought_price) * 100
            row_str = f"{item.name[:28]:<30} | {price:<10.2f} | {pnl_pct:+.2f}%"
            print(row_str)
            logging.info(row_str)
            
        total_pnl = current_equity - current_cost
        daily_pnl_change = total_pnl - previous_total_pnl
        previous_total_pnl = total_pnl
        
        sep = "-" * 50
        print(sep)
        logging.info(sep)
        
        msg_equity = f"Total Inventory Value: {current_equity:.2f}"
        msg_cost = f"Total Cost:            {current_cost:.2f}"
        msg_pnl = f"Total Net Profit:      {total_pnl:.2f}"
        msg_change = f"Daily Profit Change:   {daily_pnl_change:.2f}"
        
        print(msg_equity)
        logging.info(msg_equity)
        print(msg_cost)
        logging.info(msg_cost)
        print(msg_pnl)
        logging.info(msg_pnl)
        print(msg_change)
        logging.info(msg_change)
        
        end_sep = "=" * 50
        print(end_sep)
        logging.info(end_sep)
        
        # --- API RATE LIMIT PROTECTION ---
        print(">>> Cooling down for 5 seconds to respect API limits...")
        time.sleep(5)

print("\n=== Backtest Complete ===")
print("Final Inventory:")
print(my_inventory)

# 5. Final Settlement
print("\n=== Final Settlement Report ===")
total_value = 0.0
total_cost = 0.0
print(f"{'Item Name':<40} | {'Bought Date':<12} | {'Bought Price':<12} | {'Current Price':<12} | {'PnL %':<10}")
print("-" * 100)

for item in my_inventory.items:
    current_price = item.daily_price[-1] if item.daily_price else item.bought_price
    pnl_pct = ((current_price - item.bought_price) / item.bought_price) * 100
    total_value += current_price
    total_cost += item.bought_price
    
    p_date_str = item.purchase_date[:10] if isinstance(item.purchase_date, str) else item.purchase_date.strftime('%Y-%m-%d')
    
    print(f"{item.name[:38]:<40} | {p_date_str:<12} | {item.bought_price:<12.2f} | {current_price:<12.2f} | {pnl_pct:+.2f}%")

print("-" * 100)
print(f"Total Cost:            {total_cost:.2f}")
print(f"Total Inventory Value: {total_value:.2f}")
print(f"Total Profit/Loss:     {total_value - total_cost:.2f} ({(total_value - total_cost)/total_cost*100:.2f}%)")

2025-12-22 16:08:33,733 [INFO] ArtificialNewsAgent: Loading artificial news from cs2_trading/res/news_artificial\20251213.txt


STEAM_API_KEY loaded successfully.
Found checkpoint 'backtest_checkpoint.json'. Attempting to resume...
Resuming from 2025-12-13 (Last completed: 2025-12-12)
Loaded inventory with 10 items.
Logging configured to 'backtest.log' (mode: a)

=== Starting Budapest Major Backtest ===


>>> Processing: 2025-12-13 <<<
!!! MAJOR STARTED (Nov 24): Increasing Daily Buy Limit to 8 !!!

=== Starting Daily Cycle: 2025-12-13 ===
Step 1: Fetching News...
--- News Summary ---
--- ARTIFICIAL NEWS FOR 2025-12-13 (Found 1 files) ---

--- SOURCE: 20251213.txt ---
NAVI rally after Inferno fumble to eliminate FURIA from Budapest in instant classic
NER0cs
13-12-2025 06:35
Aleksib...
--------------------

Step 1.5: Conducting Financial Analysis...
Financial Insight: The elimination of world #1 FURIA and the underperformance of star rookie molodoy have triggered a sharp devaluation of FURIA assets, while NAVI’s upset victory has created a high-momentum "bull run" for their stickers, specifically for breakout ro

2025-12-22 16:11:29,359 [INFO] ArtificialNewsAgent: Loading artificial news from cs2_trading/res/news_artificial\20251214.txt




>>> Processing: 2025-12-14 <<<

=== Starting Daily Cycle: 2025-12-14 ===
Step 1: Fetching News...
--- News Summary ---
--- ARTIFICIAL NEWS FOR 2025-12-14 (Found 1 files) ---

--- SOURCE: 20251214.txt ---
Snax returns to GamerLegion
Sumljiv
13-12-2025 19:34
The Pole is back for a second stint in the organization.


Gam...
--------------------

Step 1.5: Conducting Financial Analysis...
Financial Insight: The market is currently reacting to speculative "legacy" narratives surrounding Snax’s return to GamerLegion and high volatility linked to s1mple’s looming contract expiration at BC.Game. Risk is **High** due to the declining competitive rankings of these organizations, which threatens the long-term price stability of their respective sticker assets. **HOLD** Snax-related items to capitalize on immediate "return" hype, but **SELL** into the volatility of BC.Game-adjacent assets before year-end contract expirations trigger a potential liquidity exit.

Step 2: Scoring Inventory...
  Bat

2025-12-22 16:12:58,132 [INFO] ArtificialNewsAgent: Loading artificial news from cs2_trading/res/news_artificial\20251215.txt


...



>>> Processing: 2025-12-18 <<<

=== Starting Daily Cycle: 2025-12-18 ===
Step 1: Fetching News...
--- News Summary ---
--- ARTIFICIAL NEWS FOR 2025-12-18 (Found 1 files) ---

--- SOURCE: 20251218.txt ---
After winning his fourth Major, apEX sets new targets
MIRAA
18-12-2025 21:30
Vitality's skipper said his team still...
--------------------

Step 1.5: Conducting Financial Analysis...
Financial Insight: The risk level is currently moderate-to-high as the market enters a post-Major "cool-down" phase and the winter break, which typically triggers profit-taking and lower liquidity despite Vitality's historic dominance. 

**HOLD** your Vitality-specific positions (Paris 2023, Austin 2025, and Budapest 2025) to capitalize on their "GOAT" era brand equity, but **SELL** generic Budapest team stickers to lock in gains before the seasonal market correction. 

While Vitality’s pursuit of a back-to-back Grand Slam in Krakow sustains long-term demand for their items, the immediate saturation of 

2025-12-22 16:19:25,268 [INFO] ArtificialNewsAgent: Loading artificial news from cs2_trading/res/news_artificial\20251219.txt




>>> Processing: 2025-12-19 <<<

=== Starting Daily Cycle: 2025-12-19 ===
Step 1: Fetching News...
--- News Summary ---
--- ARTIFICIAL NEWS FOR 2025-12-19 (Found 1 files) ---

--- SOURCE: 20251219.txt ---
magixx and zont1x return to Spirit roster as chopper and zweih are benched
MIRAA
19-12-2025 00:01
The Russian team ...
--------------------

Step 1.5: Conducting Financial Analysis...
Financial Insight: 1. The market risk is **High**, as the benching of a legendary, Major-winning IGL like chopper creates massive uncertainty regarding Spirit’s tactical stability and future trophy contention.

2. **Sell** chopper stickers immediately to capitalize on the "legacy" price spike typical of veteran benchings, and liquidate zweih assets as his value as a prospect will likely crater without active Tier-1 exposure.

3. **Hold** core donk and sh1ro signatures for long-term stability, while placing speculative **Buy** orders on magixx stickers before the 2026 BLAST season to front-run potential 

2025-12-22 16:21:05,546 [INFO] ArtificialNewsAgent: Loading artificial news from cs2_trading/res/news_artificial\20251220.txt




>>> Processing: 2025-12-20 <<<

=== Starting Daily Cycle: 2025-12-20 ===
Step 1: Fetching News...
--- News Summary ---
--- ARTIFICIAL NEWS FOR 2025-12-20 (Found 1 files) ---

--- SOURCE: 20251220.txt ---
HLTV Awards: Team of the Year finalists revealed
king_dempz
20-12-2025 02:00
Three teams stood out from the rest ac...
--------------------

Step 1.5: Conducting Financial Analysis...
Financial Insight: Vitality’s historic dominance, including back-to-back Major titles and seven consecutive event wins, has likely driven their sticker valuations to a speculative ceiling ahead of the HLTV Awards. The market risk level is **Moderate-High**, as the current hype surrounding Team of the Year finalists often precedes a "sell the news" correction during the year-end player break. My recommendation is to **HOLD** Vitality and MOUZ stickers to capture the final awards-season premium, but **SELL** Spirit and mid-tier winners like FURIA or Falcons to liquidate positions before the January volume